# Demo Run

In [1]:
%load_ext autoreload
%autoreload 2
# this may cause DisentenglementTargetManager to reimported, losing all the data e.g. configurations.

In [2]:
%matplotlib inline
%config InlineBackend.figure_format='retina'
import matplotlib.pyplot as plt
plt.rcParams["font.family"] = "DeJavu Serif"
plt.rcParams["font.serif"] = ["Times New Roman"]

In [3]:
import os
import sys
import warnings
import anndata as ad
import scanpy as sc
import torch
from pathlib import Path

sys.path.append(str(Path(os.getcwd()).resolve().parents[0]))

import tardis

local_run = True
if local_run:
    tardis.config = tardis.config_local
else:
    tardis.config = tardis.config_server
print(f"CUDA used: {torch.cuda.is_available()}")

CUDA used: False


In [4]:
adata_file_path = os.path.join(tardis.config.io_directories["processed"], "dataset_subset_sample_status_1.h5ad")
assert os.path.isfile(adata_file_path), f"File not already exist: `{adata_file_path}`"
adata = ad.read_h5ad(adata_file_path)

In [12]:
disentenglement_targets_configurations=[
    dict(
        obs_key = "integration_sample_status",
        n_reserved_latent = 3,
        counteractive_minibatch_settings = dict(
            method = "random",
            method_kwargs = dict(
                within_labels = True,
                within_batch = True,
                within_categorical_covs = [True, False],
                within_other_groups = True,
                seed = "gglobal",
            )
        ),
        auxillary_losses = dict(
            reserved_subset = dict(
                apply = True, 
                method = "wasserstein", 
                weight = 12.0, 
                negative_sign = True, 
                method_kwargs = {}
            ),
            unreserved_subset = dict(
                apply = False, 
                method = "wasserstein", 
                weight = 2.0, 
                negative_sign = True, 
                method_kwargs = {}
            ),
            complete_latent = dict(
                apply = True, 
                method = "wasserstein", 
                weight = 11.0, 
                negative_sign = True, 
                method_kwargs = {}
            ),
        ),
    ),
    dict(
        obs_key = "sample_ID",
        n_reserved_latent = 5,
        counteractive_minibatch_settings = dict(
            method = "random",
            method_kwargs = dict(
                within_labels = True,
                within_batch = True,
                within_categorical_covs = [True, False],
                within_other_groups = True,
                seed = "gglobal",
            )
        ),
        auxillary_losses = dict(
            reserved_subset = dict(
                apply = True, 
                method = "wasserstein", 
                weight = 91.0, 
                negative_sign = True, 
                method_kwargs = {}
            ),
            unreserved_subset = dict(
                apply = False, 
                method = "wasserstein", 
                weight = 92.0, 
                negative_sign = True, 
                method_kwargs = {}
            ),
            complete_latent = dict(
                apply = True, 
                method = "wasserstein", 
                weight = 93.0, 
                negative_sign = True, 
                method_kwargs = {}
            ),
        ),
    )
]

model_params = dict(
    n_hidden=256,
    n_layers=3, 
    n_latent=20, 
    gene_likelihood="nb",
    dropout_rate = 0.1
)
train_params = dict(
    max_epochs=3,
    train_size=0.1
)
dataset_params = dict(
    layer=None, 
    labels_key="cell_type",
    batch_key="concatenated_integration_covariates",
    categorical_covariate_keys=["sex", "age"],
    disentenglement_targets_configurations=disentenglement_targets_configurations
)

tardis.MyModel.setup_anndata(adata, **dataset_params)

# tardis.MyModel.setup_wandb(
#     wandb_configurations=tardis.config.wandb,
#     hyperparams=dict(
#         model_params=model_params,
#         train_params=train_params,
#         dataset_params=dataset_params,
#     )
# )

vae = tardis.MyModel(adata, **model_params)
vae.train(**train_params)

GPU available: True (mps), used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/Users/kemalinecik/tools/apps/mamba/envs/tardis_env/lib/python3.10/site-packages/lightning/pytorch/trainer/setup.py:201: UserWarning: MPS available but not used. Set `accelerator` and `devices` using `Trainer(accelerator='mps', devices=1)`.
  rank_zero_warn(
/Users/kemalinecik/tools/apps/mamba/envs/tardis_env/lib/python3.10/site-packages/lightning/pytorch/loops/fit_loop.py:281: PossibleUserWarning: The number of training batches (9) is smaller than the logging interval Trainer(log_every_n_steps=10). Set a lower value for log_every_n_steps if you want to see logs for the training epoch.
  rank_zero_warn(


Epoch 1/3:   0%|          | 0/3 [00:00<?, ?it/s]

/Users/kemalinecik/git_nosync/tardis/tardis/_counteractiveminibatchgenerator.py:60: UserWarning: Possible group definition indices are calculating for `integration_sample_status`.
  possible_indices = CachedPossibleGroupDefinitionIndices.get(
/Users/kemalinecik/git_nosync/tardis/tardis/_counteractiveminibatchgenerator.py:60: UserWarning: Number of elements in each group for `integration_sample_status`: 30,32,170,275,13,54,67,487
  possible_indices = CachedPossibleGroupDefinitionIndices.get(
/Users/kemalinecik/git_nosync/tardis/tardis/_counteractiveminibatchgenerator.py:60: UserWarning: Possible group definition indices are calculating for `sample_ID`.
  possible_indices = CachedPossibleGroupDefinitionIndices.get(
/Users/kemalinecik/git_nosync/tardis/tardis/_counteractiveminibatchgenerator.py:60: UserWarning: Number of elements in each group for `sample_ID`: 13,54,67,487,30,32,170,275
  possible_indices = CachedPossibleGroupDefinitionIndices.get(


[0, 1, 2]
[0, 1, 2]
[3, 4, 5, 6, 7]
[3, 4, 5, 6, 7]
[0, 1, 2]
[0, 1, 2]
[3, 4, 5, 6, 7]
[3, 4, 5, 6, 7]
[0, 1, 2]
[0, 1, 2]
[3, 4, 5, 6, 7]
[3, 4, 5, 6, 7]
[0, 1, 2]
[0, 1, 2]
[3, 4, 5, 6, 7]
[3, 4, 5, 6, 7]
[0, 1, 2]
[0, 1, 2]
[3, 4, 5, 6, 7]
[3, 4, 5, 6, 7]
[0, 1, 2]
[0, 1, 2]
[3, 4, 5, 6, 7]
[3, 4, 5, 6, 7]
[0, 1, 2]
[0, 1, 2]
[3, 4, 5, 6, 7]
[3, 4, 5, 6, 7]
[0, 1, 2]
[0, 1, 2]
[3, 4, 5, 6, 7]
[3, 4, 5, 6, 7]
[0, 1, 2]
[0, 1, 2]
[3, 4, 5, 6, 7]
[3, 4, 5, 6, 7]
Epoch 2/3:  33%|███▎      | 1/3 [00:00<00:00,  2.27it/s, v_num=1, train_loss_step=204, train_loss_epoch=200][0, 1, 2]
[0, 1, 2]
[3, 4, 5, 6, 7]
[3, 4, 5, 6, 7]
[0, 1, 2]
[0, 1, 2]
[3, 4, 5, 6, 7]
[3, 4, 5, 6, 7]
[0, 1, 2]
[0, 1, 2]
[3, 4, 5, 6, 7]
[3, 4, 5, 6, 7]
[0, 1, 2]
[0, 1, 2]
[3, 4, 5, 6, 7]
[3, 4, 5, 6, 7]
[0, 1, 2]
[0, 1, 2]
[3, 4, 5, 6, 7]
[3, 4, 5, 6, 7]
[0, 1, 2]
[0, 1, 2]
[3, 4, 5, 6, 7]
[3, 4, 5, 6, 7]
[0, 1, 2]
[0, 1, 2]
[3, 4, 5, 6, 7]
[3, 4, 5, 6, 7]
[0, 1, 2]
[0, 1, 2]
[3, 4, 5, 6, 7]
[3, 4, 5, 6, 7]
[0, 1, 2

`Trainer.fit` stopped: `max_epochs=3` reached.


Epoch 3/3: 100%|██████████| 3/3 [00:01<00:00,  2.87it/s, v_num=1, train_loss_step=173, train_loss_epoch=174]


__Note: Debugging takes significant amount of time__

In [6]:
# vae.plot_training_history(
#     ignore_first=0, 
#     metrics_name=["reconstruction_loss", "kl_local", "elbo"],
#     metrics_title=["Reconstruction Loss", "KL Loss", "ELBO"]
# )

In [7]:
# vae.plot_latent_kde(
#     adata_obs=adata.obs,
#     target_obs_key="sample_ID",
#     latent_representation=vae.get_latent_representation(),
#     latent_dim_of_interest=None
# )

In [8]:
# vae.plot_training_history(
#     ignore_first=0, 
#     metrics_name=[
#         "integration_sample_status|complete_latent", 
#         "integration_sample_status|reserved_subset", 
#         "integration_sample_status|unreserved_subset"
#     ],
# )

# Playground

dict_keys(['train_loss_epoch', 'elbo_train', 'reconstruction_loss_train', 'kl_local_train', 'kl_global_train', 'integration_sample_status|complete_latent_train', 'integration_sample_status|reserved_subset_train', 'integration_sample_status|unreserved_subset_train', 'sample_ID|complete_latent_train', 'sample_ID|reserved_subset_train', 'sample_ID|unreserved_subset_train', 'kl_weight', 'train_loss_step'])

In [ ]:
vae.history.

In [6]:
from tardis._disentenglementtargetmanager import DisentenglementTargetManager

In [11]:
DisentenglementTargetManager.configurations.items[1].unreserved_latent_indices

[0, 1, 2, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19]

In [13]:
DisentenglementTargetManager.configurations.items[0].reserved_latent_indices

[0, 1, 2]

## Visualization

In [ ]:
# adata.obsm["X_scVI"] = vae.get_latent_representation()
# sc.pp.neighbors(adata, n_neighbors = 30, use_rep="X_scVI")
# sc.tl.umap(adata, min_dist=0.2)

In [ ]:
# with warnings.catch_warnings():
#     warnings.simplefilter("ignore")
#     sc.pl.umap(
#         adata, 
#         color=["integration_sample_status", "sample_ID", "cell_type", "concatenated_integration_covariates"], 
#         ncols=3,
#         frameon=False,
#         title="",
#         legend_fontsize="xx-small"
#     )